In [43]:
pip install cupy-cuda115

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import cupy as cp
from numba import cuda

## Getting device information
Additional GPU properties can be found as detailed in this stackoverflow thread: https://stackoverflow.com/questions/62457151/access-gpu-hardware-specifications-in-python. This GPU has 16 GB of RAM.

In [2]:
cuda.detect()

Found 1 CUDA devices
id 0    b'NVIDIA GeForce GTX 1060 6GB'                              [SUPPORTED]
                      compute capability: 6.1
                           pci device id: 0
                              pci bus id: 6
Summary:
	1/1 devices are supported


True

## Sending data to the GPU

In [3]:
array_cpu = np.random.randint(0, 255, size=(4000, 4000))

In [4]:
array_cpu.nbytes / 1e6

64.0

In [5]:
array_gpu = cp.asarray(array_cpu)

In [6]:
%%timeit

cp.asarray(array_cpu)

7.61 ms ± 204 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
type(array_gpu)

cupy._core.core.ndarray

In [8]:
from scipy import fft

In [9]:
%%timeit

fft.fftn(array_cpu)

445 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
from cupyx.scipy import fft as fft_gpu

In [11]:
%%timeit

fft_gpu.fftn(array_gpu)

102 µs ± 29.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
fft_cpu = fft.fftn(array_cpu)
fft_sent_back = cp.asnumpy(fft_gpu.fftn(array_gpu))
np.allclose(fft_sent_back, fft_cpu)

True

In [14]:
# this will fail
fft.fftn(array_gpu)

TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.

In [15]:
# this will also fail
fft_gpu.fftn(array_cpu)

TypeError: The input array a must be a cupy.ndarray

In [16]:
# some numpy functions may work
np.max(array_gpu)

array(254, dtype=int32)

In [17]:
%%time

# if at all possible, create arrays directly on the GPU
random_gpu_array = cp.random.randint(0, 255, size=(100, 100))

Wall time: 2.34 s


## Custom kernels with numba

### Numba device arrays

In [18]:
# numba has its own API for transfering data
d_ary = cuda.to_device(array_cpu)
d_ary

In [19]:
cp.asarray(d_ary)

array([[ 91,   2, 189, ...,   5, 145, 127],
       [139,  27,  53, ...,  51,  59, 163],
       [ 16,  44, 148, ...,  14,  16,  41],
       ...,
       [205, 110, 110, ..., 200,  52, 131],
       [ 86, 246, 159, ..., 178,  78, 143],
       [175, 142,  77, ...,  25, 150,  13]])

In [20]:
d_ary.copy_to_host()

array([[ 91,   2, 189, ...,   5, 145, 127],
       [139,  27,  53, ...,  51,  59, 163],
       [ 16,  44, 148, ...,  14,  16,  41],
       ...,
       [205, 110, 110, ..., 200,  52, 131],
       [ 86, 246, 159, ..., 178,  78, 143],
       [175, 142,  77, ...,  25, 150,  13]])

### Numba kernels

In [21]:
from numba import cuda
import cupy as cp
import numpy as np

In [22]:
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)  
    if i < C.shape[0] and j < C.shape[1]:   # grid can extend beyond C
        tmp = 0.                            
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]        # multiply elements in row i of A and column j of B and add to temp
        C[i, j] = tmp

In [23]:
cp.random.seed(42)
A = cp.random.uniform(1, 10, size=(2000, 2000), dtype=np.float64)  # array 1
B = cp.random.uniform(1, 10, size=(2000, 2000), dtype=np.float64)  # array 2
C = cp.zeros((2000, 2000), dtype=np.float64)       # array where we store answer 

In [24]:
C

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
threadsperblock = (16, 16)  # each block will contain 16x16 threads, typically 128 - 512 threads/block
blockspergrid_x = int(np.ceil(C.shape[0] / threadsperblock[0]))
blockspergrid_y = int(np.ceil(C.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)  # we calculate the gridsize (number of blocks) from array
print(blockspergrid)
print(f"The kernel will be executed up to element {threadsperblock[0]*blockspergrid_x}")

(125, 125)
The kernel will be executed up to element 2000


In [26]:
# execution of the kernel
matmul[blockspergrid, threadsperblock](A, B, C)

In [27]:
C

array([[59394.46607842, 58001.66377549, 58910.89964126, ...,
        58755.23643036, 59265.65525416, 58447.86197932],
       [59656.82462269, 58635.04995946, 59080.54393462, ...,
        59327.90030958, 60391.24930458, 59425.35827899],
       [62192.77335924, 60700.17680915, 60538.34933653, ...,
        61027.03460329, 61711.10155029, 60544.69882075],
       ...,
       [60649.27416407, 59951.20972379, 60170.2004206 , ...,
        60203.88074659, 60934.19598791, 59613.28418599],
       [61620.11922557, 61264.33868343, 62076.33462258, ...,
        61227.57661876, 62642.97523374, 61841.46799761],
       [61535.95697543, 59600.43760873, 59927.620961  , ...,
        60738.55627077, 61429.70009593, 59662.34901713]])

In [28]:
# faster multiplication can be obtained by making use of shared memory between threads in the same block
# this requires more thinking about non-obvious implementation!

from numba import float32, int32, float64

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [29]:
# execution of the kernel
SIZE = 2000
A = cp.random.uniform(1, 10, size=(SIZE, SIZE), dtype=np.float32)
B = cp.random.uniform(1, 10, size=(SIZE, SIZE), dtype=np.float32)
C_slow = cp.zeros((SIZE, SIZE), dtype=np.float32)      
C_fast = cp.zeros((SIZE, SIZE), dtype=np.float32)

In [30]:
threadsperblock = (TPB, TPB)
blockspergrid = int(np.ceil(SIZE / threadsperblock[0]))
blockspergrid = (blockspergrid, blockspergrid)

In [31]:
matmul[blockspergrid, threadsperblock](A, B, C_slow)

In [32]:
fast_matmul[blockspergrid, threadsperblock](A, B, C_fast)

In [33]:
cp.allclose(C_slow, C_fast)

array(True)

In [34]:
%%time

for i in range(10):
  matmul[blockspergrid, threadsperblock](A, B, C_slow)

Wall time: 2.45 s


In [35]:
%%time

for i in range(10):
  fast_matmul[blockspergrid, threadsperblock](A, B, C_fast)

Wall time: 3.74 s


In [36]:
C_c = cp.dot(A, B)

In [37]:
np.allclose(C_c, C_fast)

array(True)

In [38]:
%time

for i in range(10):
  cp.dot(A, B)

Wall time: 0 ns


In [39]:
del 

SyntaxError: invalid syntax (<ipython-input-39-d8959d277f58>, line 1)